In [ ]:
import os 
import numpy as np
import pandas as pd
from tensorflow import keras

In [ ]:
os.environ['KAGGLE_USERNAME'] = "davidfmora"
os.environ['KAGGLE_KEY'] = "204c0028a80148851fdab46383061779"

! kaggle competitions download -c titanic # Descarga del dataset

  0% 0.00/28.0k [00:00<?, ?B/s]
100% 28.0k/28.0k [00:00<00:00, 26.6MB/s]
  0% 0.00/3.18k [00:00<?, ?B/s]
100% 3.18k/3.18k [00:00<00:00, 3.93MB/s]
  0% 0.00/59.8k [00:00<?, ?B/s]
100% 59.8k/59.8k [00:00<00:00, 59.8MB/s]


In [ ]:
# datos de entrenamiento
train_data = pd.read_csv("train.csv")
# datos de evaluacion
test_data = pd.read_csv("test.csv")

In [ ]:
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
features = ["Sex", "SibSp", "Pclass", "Parch"]
# Codificacion de las caracteristicas a OneHot
train_x = pd.get_dummies(train_data[features]).to_numpy()
test_x = pd.get_dummies(test_data[features]).to_numpy()
# etiquedas casteadas a float
train_y = train_data["Survived"].to_numpy().astype("float32")



In [ ]:
from sklearn.model_selection import train_test_split
# Separacion de datos de validacion y entrenamiento
train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=0.33)

In [ ]:
from keras import models, layers

# red densa
model = models.Sequential()
model.add(layers.Dense(16,activation='relu', input_shape=(5,))) 
model.add(layers.Dropout(0.5))
model.add(layers.Dense(16,activation='relu'))
model.add(layers.Dense(1,activation='sigmoid'))

model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy']) 

In [ ]:
# entrenamiento a 80 epochs
model.fit(train_x, train_y, batch_size=32, epochs=80, verbose=0)

In [ ]:
# evaluacion en los datos de validacion 
model.evaluate(val_x, val_y)

10/10 [==============================] - 0s 2ms/step - loss: 0.4685 - accuracy: 0.8102


[0.4684853255748749, 0.810169517993927]

In [ ]:
# Predicciones en probabilidades
raw_predictions = model.predict(val_x).tolist()
# Predicciones redondeadas
predictions = [float(round(x[0])) for x in raw_predictions]
# porcentaje de precision en los datos de validacion
correct = sum([int(x == y) for x, y in zip(val_y, predictions)])/len(predictions)
correct

0.7966101694915254

In [ ]:
# predicciones en los datos de testeo
raw_predictions = model.predict(test_x).tolist()
predictions = [round(x[0]) for x in raw_predictions]

# csv de entrega
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('submission.csv', index=False)

In [ ]:
! kaggle competitions submit -c titanic -f submission.csv -m "Titanic"

100% 2.77k/2.77k [00:03<00:00, 867B/s]
Successfully submitted to Titanic - Machine Learning from Disaster